In [ ]:
!git clone https://github.com/BoPDA1607/Vietnamese_TTS_Finetune

Cloning into 'Vietnamese_TTS_Finetune'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 42 (delta 7), reused 33 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 5.98 MiB | 12.07 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
%cd /content/Vietnamese_TTS_Finetune

/content/Vietnamese_TTS_Finetune


In [ ]:
!apt install espeak-ng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-1

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 28.9 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      Successfully uninstalled torchao-0.10.0


In [ ]:
!pip install transformers datasets torch soundfile flask-cors pyngrok scipy

In [ ]:
!ngrok config add-authtoken 2mNDk6a1ZBoOk8bIK4EmmyHHfFK_5RvWSgNtVtcjVsqf9nL1d

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# 3. Import các thư viện
import threading
import io
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
import torch
import torch.hub
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from vietnamese_tts import VietnameseTTS
from utils.normalize_text import VietnameseTTSNormalizer
import soundfile as sf

In [ ]:
# 4. Khởi tạo Flask App
app = Flask(__name__)
CORS(app)
port = "5000"

In [ ]:
# 5. Tải mô hình TTS (Text-to-Speech) trên toàn cục
print("Đang tải mô hình TTS, việc này sẽ mất một chút thời gian...")

device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    tts = VietnameseTTS(
        backbone_repo="pnnbao-ump/VieNeu-TTS",
        backbone_device=device,
        codec_repo="neuphonic/neucodec",
        codec_device=device
    )
    normalizer = VietnameseTTSNormalizer()

except Exception as e:
    print(f"❌ Lỗi khi tải mô hình: {e}")
    tts = None # Đặt là None để API có thể báo lỗi

Đang tải mô hình TTS, việc này sẽ mất một chút thời gian...
Loading backbone from: pnnbao-ump/VieNeu-TTS on cuda ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/24.1M [00:00<?, ?B/s]

...

meta.yaml:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
# 6. Tạo API Endpoint cho việc tổng hợp giọng nói

@app.route("/synthesize", methods=['POST'])
def synthesize_speech():
    """
    Nhận văn bản (text) từ JSON, tạo file âm thanh và gửi trả lại.
    """
    if not tts:
        return jsonify({'error': 'Mô hình TTS chưa được tải.'}), 500

    # 1. Lấy text từ request
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({'error': 'Không tìm thấy key "text" trong JSON payload.'}), 400

    text_input = data['text']
    voice_selection = data['voice']

    try:
        ref_audio = f"./sample/{voice_selection}.wav"
        ref_text = Path(f"./sample/{voice_selection}.txt").read_text(encoding="utf-8")

        ref_text_norm = normalizer.normalize(ref_text)
        ref_codes = tts.encode_reference(ref_audio)

        inputs = normalizer.normalize(text_input)

        print(f"Đang tổng hợp: '{text_input}'...")

        speech = tts.infer(inputs, ref_codes, ref_text_norm)

        print("Tổng hợp hoàn tất.")

        # 3. Lưu âm thanh vào bộ đệm (memory) thay vì lưu file
        buffer = io.BytesIO()
        # Ghi dữ liệu âm thanh vào buffer dưới dạng file WAV

        wavfile.write(buffer, 24000, speech)
        buffer.seek(0) # Đưa con trỏ về đầu buffer

        # 4. Gửi file âm thanh về cho client
        return send_file(
            buffer,
            mimetype="audio/wav",
            as_attachment=True,
            download_name="output.wav"
        )

    except Exception as e:
        print(f"Lỗi trong quá trình tổng hợp: {e}")
        return jsonify({"error": f"Lỗi server: {str(e)}"}), 500


In [ ]:
# 7. Chạy ứng dụng
print("Đang mở ngrok tunnel...")
# Mở một tunnel ngrok
public_url = ngrok.connect(port).public_url
print(f" * Ngrok Tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
print(f" * Endpoint của bạn là: {public_url}/synthesize")
print(" * Gửi yêu cầu POST với JSON вида {'text': 'hello world'} đến endpoint trên.")

app.config["BASE_URL"] = public_url

# Chạy Flask server trong một thread riêng
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

Đang mở ngrok tunnel...
 * Ngrok Tunnel "https://7fcf7c28ca35.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Endpoint của bạn là: https://7fcf7c28ca35.ngrok-free.app/synthesize
 * Gửi yêu cầu POST với JSON вида {'text': 'hello world'} đến endpoint trên.
